In [1]:
import numpy as np
import awkward as ak
import uproot_methods

In [2]:
import logging
logging.basicConfig(level=logging.INFO, format='[%(asctime)s] %(levelname)s: %(message)s')

In [3]:
def stack_arrays(a, keys, axis=-1):
    flat_arr = np.stack([a[k].flatten() for k in keys], axis=axis)
    return awkward.JaggedArray.fromcounts(a[keys[0]].counts, flat_arr)

In [4]:
def pad_array(a, maxlen, value=0., dtype='float32'):
    x = (np.ones((len(a), maxlen)) * value).astype(dtype)
    for idx, s in enumerate(a):
        if not len(s):
            continue
        trunc = s[:maxlen].astype(dtype)
        x[idx, :len(trunc)] = trunc
    return x

In [27]:
##and Professor suggests that we could use mass, classifacation for later application
def SetAKArr(filepath):
    with open(filepath, 'r') as file:
        lines = file.readlines()
    n_particles_ls = []
    px_ls = []
    py_ls = []
    pz_ls = []
    energy_ls = []
    mass_ls = []
    charge_ls = []
    _label1 = []
    _label2 = []
    _label3 = []
    _label4 = []
    _label5 = []
    
    n = 0
    for line in lines:
        if line.startswith('E'):
            if not n == 0:
                n_particles_ls.append(n)
                n = 0
            exp_inf = line.split()
#             _label1.append(int(exp_inf[1]))
#             _label2.append(1-int(exp_inf[1]))
#             _label1.append(1)
#             _label2.append(0)
            _label1.append(float(exp_inf[1]))
            _label2.append(float(exp_inf[2]))
            _label3.append(float(exp_inf[3]))
            _label4.append(float(exp_inf[4]))
            _label5.append(float(exp_inf[5]))
        else:
            par = line.split()
            ##particle +1
            n = n + 1
            px_ls.append(abs(float(par[2])))
            py_ls.append(abs(float(par[3])))
            pz_ls.append(abs(float(par[4])))
            energy_ls.append(abs(float(par[5])))
            mass_ls.append(abs(float(par[6])))  
            charge_ls.append(int(par[0]))
#             px_ls.append(6)
#             py_ls.append(2)
#             pz_ls.append(3)
#             energy_ls.append(4)
#             mass_ls.append(5)
    if not n == 0:
        n_particles_ls.append(n)
    px_arr = np.array(px_ls)
    py_arr = np.array(py_ls)
    pz_arr = np.array(pz_ls)
    energy_arr = np.array(energy_ls)
    mass_arr = np.array(mass_ls)
    charge_arr = np.array(charge_ls)
    n_particles = np.array(n_particles_ls)

#     print(n_particles)
    px = ak.JaggedArray.fromcounts(n_particles, px_arr)
    py = ak.JaggedArray.fromcounts(n_particles, py_arr)
    pz = ak.JaggedArray.fromcounts(n_particles, pz_arr)
    energy = ak.JaggedArray.fromcounts(n_particles, energy_arr)
    mass = ak.JaggedArray.fromcounts(n_particles, mass_arr)
    charge = ak.JaggedArray.fromcounts(n_particles, charge_arr)
    p4 = uproot_methods.TLorentzVectorArray.from_cartesian(px, py, pz, energy)
    ##Create an Order Dic
    from collections import OrderedDict
    v = OrderedDict()
    v['part_px'] = px
#     print(px)
    v['part_py'] = py
    v['part_pz'] = pz
    v['part_energy'] = energy
    v['part_mass'] = mass
    v['charge'] = charge
    v['part_e_log'] = np.log(energy)
    v['part_px_log'] = np.log(px)
    v['part_py_log'] = np.log(py)
    v['part_pz_log'] = np.log(pz)
    v['part_m_log'] = np.log(mass)
#     ls1 = [1,2,3,4]
#     ls2 = [5,6,7,8]
#     v['label'] = np.stack((_label1, _label2, _label3, _label4, _label5), axis=-1)
#     print(v['label'])
    v['label'] = np.stack((_label1, _label2, _label3, _label4, _label5), axis = -1)
#     v['label'] = np.stack(_label1, axis = -1)
#     print(v['label'])
    return v

In [28]:
class Dataset(object):
    def __init__(self, filepath, feature_dict = {}, label = 'label', pad_len=100, data_format='channel_first'):
        self.filepath = filepath
        self.feature_dict = feature_dict
        if len(feature_dict) == 0:
            feature_dict['points'] = ['part_px','part_py','part_pz']
            feature_dict['features'] = ['part_energy', 'part_mass', 'charge', 'part_px', 'part_py', 'part_pz']
            feature_dict['mask'] = ['part_energy']
        ##currently we use 'E' for experiments
        self.label = label
        self.pad_len = pad_len
        assert data_format in ('channel_first', 'channel_last')
        self.stack_axis = 1 if data_format=='channel_first' else -1
        self._values = {}
        self._label = None
        self._load()
        
    def _load(self):
        logging.info('Start loading file %s' % self.filepath)
#         counts = None
        a = SetAKArr(self.filepath)
        self._label = a[self.label]
        for k in self.feature_dict:
                cols = self.feature_dict[k]
                if not isinstance(cols, (list, tuple)):
                    cols = [cols]
                arrs = []
                for col in cols:
                    arrs.append(pad_array(a[col], self.pad_len))
                    ##check the dimesion of a[col], and it should be array.
                self._values[k] = np.stack(arrs, axis=self.stack_axis)
        logging.info('Finished loading file %s' % self.filepath)
        
        
        
    def __len__(self):
        return len(self._label)

    def __getitem__(self, key):
        if key==self.label:
            return self._label
        else:
            return self._values[key]
    
    @property
    def X(self):
        return self._values
    
    @property
    def y(self):
        return self._label

    def shuffle(self, seed=None):
        if seed is not None:
            np.random.seed(seed)
        shuffle_indices = np.arange(self.__len__())
        np.random.shuffle(shuffle_indices)
        for k in self._values:
            self._values[k] = self._values[k][shuffle_indices]
        self._label = self._label[shuffle_indices]

In [29]:
train_dataset = Dataset('train.txt', data_format='channel_last')
val_dataset = Dataset('val.txt', data_format='channel_last')
test_dataset = Dataset('test.txt', data_format = 'channel_last')

[2024-03-02 14:41:32,856] INFO: Start loading file train.txt
[2024-03-02 14:41:33,042] INFO: Finished loading file train.txt
[2024-03-02 14:41:33,043] INFO: Start loading file val.txt
[2024-03-02 14:41:33,069] INFO: Finished loading file val.txt
[2024-03-02 14:41:33,070] INFO: Start loading file test.txt
[2024-03-02 14:41:33,097] INFO: Finished loading file test.txt


In [30]:
import tensorflow as tf
from tensorflow import keras
from tf_keras_model import get_particle_net, get_particle_net_lite

In [31]:
model_type = 'particle_net_lite' # choose between 'particle_net' and 'particle_net_lite'
##this shows the number of classes for classification
num_classes = train_dataset.y.shape[1]
# num_classes = 1
# print(num_classes)
input_shapes = {k:train_dataset[k].shape[1:] for k in train_dataset.X}
if 'lite' in model_type:
    model = get_particle_net_lite(num_classes, input_shapes)
else:
    model = get_particle_net(num_classes, input_shapes)

In [32]:
# Training parameters
batch_size = 1024 if 'lite' in model_type else 384
epochs = 30

In [33]:
def lr_schedule(epoch):
    lr = 1e-3
    if epoch > 10:
        lr *= 0.1
    elif epoch > 20:
        lr *= 0.01
    logging.info('Learning rate: %f'%lr)
    return lr

In [47]:
# model.compile(loss='categorical_crossentropy',
#               optimizer=keras.optimizers.Adam(learning_rate=lr_schedule(0)),
#               metrics=['accuracy'])
model.compile(loss='log_cosh',
              optimizer=keras.optimizers.Adam(learning_rate=lr_schedule(0)),
              metrics=['accuracy'])
model.summary()

[2024-03-02 15:00:15,986] INFO: Learning rate: 0.001000


Model: "ParticleNet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
mask (InputLayer)               [(None, 100, 1)]     0                                            
__________________________________________________________________________________________________
tf_op_layer_NotEqual_2 (TensorF [(None, 100, 1)]     0           mask[0][0]                       
__________________________________________________________________________________________________
tf_op_layer_Cast_4 (TensorFlowO [(None, 100, 1)]     0           tf_op_layer_NotEqual_2[0][0]     
__________________________________________________________________________________________________
tf_op_layer_Equal_2 (TensorFlow [(None, 100, 1)]     0           tf_op_layer_Cast_4[0][0]         
________________________________________________________________________________________

In [48]:
# Prepare model model saving directory.
import os
save_dir = 'model_checkpoints'
model_name = '%s_model.{epoch:03d}.h5' % model_type
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
filepath = os.path.join(save_dir, model_name)

# Prepare callbacks for model saving and for learning rate adjustment.
checkpoint = keras.callbacks.ModelCheckpoint(filepath=filepath,
                             monitor='val_acc',
                             verbose=1,
                             save_best_only=True)

lr_scheduler = keras.callbacks.LearningRateScheduler(lr_schedule)
progress_bar = keras.callbacks.ProgbarLogger()
callbacks = [checkpoint, lr_scheduler]

In [49]:
train_dataset.shuffle()
model.fit(train_dataset.X, train_dataset.y,
          batch_size=batch_size,
#           epochs=epochs,
          epochs=10, # --- train only for 1 epoch here for demonstration ---
          validation_data=(val_dataset.X, val_dataset.y),
          shuffle=True,
          callbacks=callbacks)

[2024-03-02 15:00:18,036] INFO: Learning rate: 0.001000


Epoch 1/10
5/5 [==============================] - ETA: 0s - accuracy: 0.5111 - loss: 0.0388

[2024-03-02 15:00:24,367] WARNING: Can save best model only with val_acc available, skipping.


5/5 [==============================] - 4s 849ms/step - accuracy: 0.5111 - loss: 0.0388 - val_loss: 0.6287 - val_accuracy: 1.0000


[2024-03-02 15:00:24,371] INFO: Learning rate: 0.001000


...

5/5 [==============================] - 4s 838ms/step - accuracy: 0.5231 - loss: 0.0330 - val_loss: 0.4790 - val_accuracy: 1.0000


[2024-03-02 15:01:07,068] INFO: Learning rate: 0.001000


Epoch 10/10
5/5 [==============================] - ETA: 0s - accuracy: 0.5222 - loss: 0.0327

[2024-03-02 15:01:12,361] WARNING: Can save best model only with val_acc available, skipping.


5/5 [==============================] - 4s 819ms/step - accuracy: 0.5222 - loss: 0.0327 - val_loss: 0.4561 - val_accuracy: 0.9979


In [50]:
predictions = model.predict(test_dataset.X)
for prediction in predictions:
    print(prediction)

[ 3.4795888e-03 -1.0228133e-02  9.2060687e-03  3.5541360e+00
  3.5120909e+00]
[-8.3507560e-03  8.8749267e-04  1.4569951e-02  3.6014533e+00
  3.5806708e+00]
[-1.9754730e-03  1.5933985e-02  1.4940263e-02  3.4935634e+00
  3.4604492e+00]
[0.0076039  0.01390027 0.01716801 3.5046504  3.4753296 ]
[0.00422866 0.00566999 0.01364815 3.557429   3.5238142 ]
[ 2.2510625e-03 -9.5686149e-03  2.6632600e-02  3.4791803e+00
  3.4524286e+00]
[5.7180598e-04 1.9145386e-02 2.2042612e-02 3.4536324e+00 3.4301333e+00]
[3.0225143e-04 5.2259583e-03 1.1660975e-02 3.5273192e+00 3.4971416e+00]
[-0.00405788  0.0047426   0.01489033  3.5780785   3.5455964 ]
[0.01015927 0.02483952 0.02603246 3.469682   3.4396493 ]
[ 1.3154302e-02 -1.5272107e-03  2.2572363e-02  3.5513060e+00
  3.5126300e+00]
[9.0786442e-04 2.4343524e-03 9.6832747e-03 3.5516460e+00 3.5251667e+00]
[-3.3085905e-03  9.5469225e-03  1.2995491e-02  3.4207766e+00
  3.3926041e+00]
[2.0297803e-03 1.7594771e-02 1.6882537e-02 3.4790833e+00 3.4331584e+00]
[0.00980536